<a href="https://colab.research.google.com/github/Avr8/Diabetic_Retinopathy/blob/main/DiabeticRetinopathy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Research Kernel Link - https://github.com/dimitreOliveira/APTOS2019BlindnessDetection/blob/master/Model%20backlog/ResNet50/4%20-%20ResNet50%20-%20Batch%20size%2022.ipynb

import pandas as pd
import numpy as np
import os
from prettytable import PrettyTable
import pickle
import multiprocessing
from multiprocessing.pool import ThreadPool
print(multiprocessing.cpu_count()," CPU cores")

import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams["axes.grid"] = False

from sklearn.metrics import confusion_matrix, cohen_kappa_score,accuracy_score

from PIL import Image
import cv2

import keras
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers,Model,Sequential
from keras.layers import Input,GlobalAveragePooling2D,Dropout,Dense,Activation


# Colab Libs...
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

2  CPU cores


In [ ]:
# Importing Libraries
#ref - https://buomsoo-kim.github.io/colab/2018/04/16/Importing-files-from-Google-Drive-in-Google-Colab.md/

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
os.chdir('/content/gdrive/My Drive/pakka_final')
print("We are currently in the folder of ",os.getcwd())

We are currently in the folder of  /content/gdrive/My Drive/pakka_final


In [ ]:
import zipfile
def load_data():
    data_test = pd.read_csv('TEST/test.csv')
    data_train = pd.read_csv('TRAIN/train.csv')
    
   # !unzip TRAIN/train_images.zip -d TRAIN
    # !unzip TEST/test_images.zip  -d TEST
    # zip_ref_train = zipfile.ZipFile('TRAIN/train_images.zip','r')
    # zip_ref_train.extractall()
    # zip_ref_train.close()
    # zip_ref_test = zipfile.ZipFile('TEST/test_images.zip','r')
    # zip_ref_test.extractall()
    # zip_ref_test.close()
    return 

In [ ]:
load_data()

In [ ]:
data_x = pd.read_csv('TRAIN/train.csv')
print(data_x.head(19))

         id_code  diagnosis
0   000c1434d8d7          2
1   001639a390f0          4
2   0024cdab0c1e          1
3   002c21358ce6          0
4   005b95c28852          0
5   0083ee8054ee          4
6   0097f532ac9f          0
7   00a8624548a9          2
8   00b74780d31d          2
9   00cb6555d108          1
10  00cc2b75cddd          0
11  00e4ddff966a          2
12  00f6c1be5a33          0
13  0104b032c141          3
14  0124dffecf29          1
15  0125fbd2e791          0
16  012a242ac6ff          2
17  014508ccb9cb          0
18  0151781fe50b          0


In [ ]:
import os
os.mkdir('/content/gdrive/My Drive/pakka_final/train')
os.mkdir('/content/gdrive/My Drive/pakka_final/train/No-dr')
os.mkdir('/content/gdrive/My Drive/pakka_final/train/Mild')
os.mkdir('/content/gdrive/My Drive/pakka_final/train/Moderate')
os.mkdir('/content/gdrive/My Drive/pakka_final/train/Severe')
os.mkdir('/content/gdrive/My Drive/pakka_final/train/Proliferative-dr')

In [ ]:
import shutil
from shutil import copyfile
map_dict = {0:'No-dr',1:'Mild',2:'Moderate',3:'Severe',4:'Proliferative-dr'}
for i in range(0,len(data_x)):
  img,diagnosis = data_x.iloc[i][0],data_x.iloc[i][1]
  shutil.copy('/content/gdrive/MyDrive/pakka_final/TRAIN/'+img+'.png','/content/gdrive/My Drive/pakka_final/train/'+map_dict[diagnosis])

In [ ]:
data_y = pd.read_csv('TEST/test.csv')

In [ ]:
!pip install tqdm
!pip install time


ERROR: Could not find a version that satisfies the requirement time (from versions: none)
ERROR: No matching distribution found for time


In [ ]:
directory = '/content/gdrive/My Drive/pakka_final/train/No-dr'
x_train,y_train = [],[]
for x in tqdm(os.listdir(directory)):
  dir = os.path.join(directory,x)
  frame = cv2.imread(dir)
  frame = cv2.resize(frame,(320,320))
  frame = frame.reshape(320,320,3)
  x_train.append(frame)
  y_train.append(0)
directory = '/content/gdrive/My Drive/pakka_final/train/Mild'
for x in tqdm(os.listdir(directory)):
  dir = os.path.join(directory,x)
  frame = cv2.imread(dir)
  frame = cv2.resize(frame,(320,320))
  frame = frame.reshape(320,320,3)
  x_train.append(frame)
  y_train.append(1)
directory = '/content/gdrive/My Drive/pakka_final/train/Moderate'
for x in tqdm(os.listdir(directory)):
  dir = os.path.join(directory,x)
  frame = cv2.imread(dir)
  frame = cv2.resize(frame,(320,320))
  frame = frame.reshape(320,320,3)
  x_train.append(frame)
  y_train.append(2)
directory = '/content/gdrive/My Drive/pakka_final/train/Severe'
for x in tqdm(os.listdir(directory)):
  dir = os.path.join(directory,x)
  frame = cv2.imread(dir)
  frame = cv2.resize(frame,(320,320))
  frame = frame.reshape(320,320,3)
  x_train.append(frame)
  y_train.append(3)
directory = '/content/gdrive/My Drive/pakka_final/train/Proliferative-dr'
for x in tqdm(os.listdir(directory)):
  dir = os.path.join(directory,x)
  frame = cv2.imread(dir)
  frame = cv2.resize(frame,(320,320))
  frame = frame.reshape(320,320,3)
  x_train.append(frame)
  y_train.append(4)
np.save('/content/gdrive/My Drive/pakka_final/x.npy',x_train)
np.save('/content/gdrive/My Drive/pakka_final/y.npy',y_train)

100%|██████████| 295/295 [06:09<00:00,  1.25s/it]


In [ ]:
# Model parameters
BATCH_SIZE = 8
EPOCHS = 40
WARMUP_EPOCHS = 2
LEARNING_RATE = 1e-4
WARMUP_LEARNING_RATE = 1e-3
HEIGHT = 320
WIDTH = 320
CANAL = 3
N_CLASSES = 5
ES_PATIENCE = 5
RLROP_PATIENCE = 3
DECAY_DROP = 0.5

In [ ]:
import tensorflow as tf
def create_model():
    model=tf.keras.Sequential([
    tf.keras.layers.Conv2D(32,(5,5),input_shape=(320,320,3),activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(5,activation='softmax')
    ])
    return model

In [ ]:
model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 316, 316, 32)      2432      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 158, 158, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 156, 156, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 78, 78, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 76, 76, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 38, 38, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 92416)             0

In [ ]:
x_train = np.load('x.npy')
y_train = np.load('y.npy')
x_train = np.array(x_train)
y_train = np.array(y_train)
one_hot_labels = np.zeros((len(y_train),5))
for i,l in enumerate(y_train):
  one_hot_labels[i][l] = 1
y_train = one_hot_labels
model.compile(optimizer = 'adam',loss = 'categorical_crossentropy',metrics = ['accuracy'])
history_warmup = model.fit(x_train,y_train,
                                    epochs=10).history

Epoch 1/10
115/115 [==============================] - 20s 167ms/step - loss: 0.7843 - accuracy: 0.7135
Epoch 2/10
115/115 [==============================] - 19s 166ms/step - loss: 0.8117 - accuracy: 0.7092
Epoch 3/10
115/115 [==============================] - 19s 167ms/step - loss: 0.7712 - accuracy: 0.7193
Epoch 4/10
115/115 [==============================] - 19s 166ms/step - loss: 0.7480 - accuracy: 0.7182
Epoch 5/10
115/115 [==============================] - 19s 166ms/step - loss: 0.7138 - accuracy: 0.7351
Epoch 6/10
115/115 [==============================] - 19s 165ms/step - loss: 0.6988 - accuracy: 0.7419
Epoch 7/10
115/115 [==============================] - 19s 166ms/step - loss: 0.6808 - accuracy: 0.7482
Epoch 8/10
115/115 [==============================] - 19s 165ms/step - loss: 0.6596 - accuracy: 0.7496
Epoch 9/10
 39/115 [=========>....................] - ETA: 12s - loss: 0.6347 - accuracy: 0.7700

In [ ]:
def upload_files():
  from google.colab import files
  uploaded = files.upload()
  for k, v in uploaded.items():
    open(k, 'wb').write(v)
  return list(uploaded.keys())

In [ ]:
ans = upload_files()


Saving 0af296d2f04a.png to 0af296d2f04a.png


In [ ]:
frame = cv2.imread('0bf37ca3156a.png')
frame = cv2.resize(frame,(320,320))
frame = frame.reshape(1,320,320,3)
map_dict = {0:'No-dr',1:'Mild',2:'Moderate',3:'Severe',4:'Proliferative-dr'}
ans = np.argmax(model.predict(frame)[0])
print(map_dict[ans])

No-dr
